In [3]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
####################################################################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 
# 8/23/23 We want to add:
#     Take into account the RELATIVE STIM LOCATION within every barrel. Does velocity depend on distance from stimulating location?
#            New column: distance (um) from stim loc to relevant barrel pair boundary (1:2)
#            New column: distance (um) from stim loc to relevant barrel pair boundary (2:3)
#
#     METHOD
#         To calculate stim location to boundary, take these files produced by MovieMaker2.ipynb:
#             electrode.dat
#             corners_layer.dat
#             corners_barrel.dat
#         Calculate: Distance to lower barrel, distance to higher barrel,
#                    Distance to lower layer, distance to higher layer
#                       (if applicable for each)
####################################################################################

In [4]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
cell_type_prefix = 'scnn1a'
csv_prefix = cell_type_prefix + '_all_slices.csv'
csv_columns = ['Date',
             'Slice/Loc/Rec',
             'Stim Location',
             'L4 barrel 1 Latency',
             'L4 barrel 2 Latency',
             'L4 barrel 3 Latency',
             'L4 barrel 1 SNR',
             'L4 barrel 2 SNR',
             'L4 barrel 3 SNR',
             'ROI jpg',
              'Age',
              'Infusion',
              'Sex',
              'Stim (uA)',
              'Delta latency 1 (barrel 1 - barrel 2)',
              'Delta latency 2 (barrel 3 - barrel 2)']

enable_photoZ_interaction = True
initialize_photoZ = True
snr_cutoff_signal = 5.0
pixel_to_um_conversion = 480 / 80 # um per pixel

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/" + cell_type_prefix +"_catalog.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,...,Sex,Stim (uA),Delta latency 1 (barrel 1 - barrel 2),Delta latency 2 (barrel 3 - barrel 2),abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2
0,2/28/2023,02_01_01,NaN,L5,50.3009,50.8406,49.3457,2.940390,10.630900,6.484390,...,M,100,NaN,-1.4949,NaN,1.4949,NaN,NaN,NaN,NaN
1,2/28/2023,02_02_02,NaN,L5,49.9882,51.3212,51.1805,2.045210,5.142090,2.921330,...,M,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2/28/2023,03_01_01,NaN,L2/3,52.5719,51.2136,52.9013,9.405090,10.237200,5.780930,...,M,100,1.3583,1.6877,1.3583,1.6877,NaN,NaN,NaN,NaN
3,2/28/2023,03_02_02,NaN,L2/3,51.8399,50.0310,50.7536,14.738200,35.400200,21.883200,...,M,100,1.8089,0.7226,1.8089,0.7226,NaN,NaN,NaN,NaN
4,2/28/2023,06_01_01,NaN,L5,51.1367,51.0128,50.4326,5.726470,13.058300,13.063500,...,M,100,0.1239,-0.5802,0.1239,0.5802,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,6/28/2023,02_04_03,Posterior,L4,58.3278,94.0000,94.0000,1.284740,0.000000,0.000000,...,F,200,NaN,NaN,NaN,NaN,18.240434,11.561971,18.967022,22.090312
86,6/28/2023,03_01_10,Posterior,L4,94.0000,51.1749,51.3676,0.000000,0.263371,1.110580,...,F,200,NaN,NaN,NaN,NaN,29.504248,2.526764,16.917808,9.432646
87,6/28/2023,03_02_03,Posterior,L4,58.9868,58.3472,94.0000,0.077615,0.514122,0.000000,...,F,200,NaN,NaN,NaN,NaN,14.071951,11.100574,9.111532,19.244623
88,6/28/2023,03_03_01,Posterior,L4,50.7398,94.0000,94.0000,0.862532,0.000000,0.000000,...,F,200,NaN,NaN,NaN,NaN,9.567544,32.396742,4.680122,23.260346


In [5]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [11]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'mm_hidden' in subdir:
        continue
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'layer4_barrels_slice' in f[-35:] and f.endswith('_rois.dat'):
            slice_no = int(f.split("_rois.dat")[0][-1])
            roi_files[slice_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
            
        # reach into analysis##_##_## subdir (created by MovieMaker2)
        analysis_subdir = subdir + '/analysis' + rec_id
        corners_barrel_file = analysis_subdir + "/corners_barrel.dat"
        corners_layer_file = analysis_subdir + "/corners_layer.dat"
        electrode_file = analysis_subdir + "/electrode.dat"
        
        # read in barrel and layer axes data
        barrel_layer_axes = []
        for la_file in [corners_barrel_file, corners_layer_file]:
            print("\t", la_file)
            with open(la_file, 'r') as f:
                lines = f.readlines() 
            corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
            print("\t", corners)
            layer_axes = LayerAxes(corners, verbose=False)
            print("\t", layer_axes)
            laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
            barrel_layer_axes.append(laminar_axis)
            barrel_layer_axes.append(laminar_axis_2)
        
        # open electrode as a single integer (its diode number) in variable stim_pt
        with open(electrode_file, 'r') as f:
            lines = f.readlines()
        stim_pt = int(lines[-1]) # last line is always electrode location
        aux_obj = LaminarROI([stim_pt]).get_points()
        stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
        
        # compute perpendicular distances between each of the 4 lines and the electrode point
        perp_dists = [line_obj.get_distance_to_point(stim_pt) * pixel_to_um_conversion for line_obj in barrel_layer_axes]
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = roi_files[slice_no]
        print("\n\nscraping data from ", zda_file, "using", roi_file, "on date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(subdir + "/" + roi_file)
            time.sleep(5)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)

        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from cell_type_prefix_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        # write perpendicular distances
        new_row['Distance to Barrel A'] = perp_dists[0]
        new_row['Distance to Barrel B'] = perp_dists[1]
        new_row['Distance to Layer nA'] = perp_dists[2]
        new_row['Distance to Layer nB'] = perp_dists[3]
        
        # open these various files and store them to the pandas dataframe
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
        df = df.append(new_row, ignore_index=True)


	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-18-23\selected_zda/analysis03_01_03/corners_barrel.dat
	 [5297, 666, 5655, 864]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-18-23\selected_zda/analysis03_01_03/corners_layer.dat
	 [2808, 3436, 4806, 5274]


scraping data from  03_01_03.zda using layer4_barrels_slice3_rois.dat on date: 4/18/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-18-23\selected_zda/analysis03_02_01/corners_barrel.dat
	 [5300, 586, 5733, 857]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-18-23\selected_zda/analysis03_02_01/corners_layer.dat
	 [3048, 3516, 4807, 5194]


scraping data from  03_02_01.zda using layer4_barrels_slice3_rois.dat on date: 4/18/2023


In [ ]:
# write the pandas dataframe back to csv, fill in the Stim Layer metadata, and read it back in to df
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)
input("Press Enter once you have filled in the Stim Layer metadata in " + csv_filename)

In [ ]:
# load the all-data CSV dir after you've filled in the Stim location metadata
df = pd.read_csv(data_dir + csv_filename)  # from the edited csv
df = pd.DataFrame(df)
df

In [ ]:
# compute statistics on barrel pairs:
#    1) whether delta latency should be considered based on SNR
#    2) Delta latency between L4 barrels (b1 - b2, b3 - b2)
df['Delta latency 1 (barrel 1 - barrel 2)'] = df['L4 barrel 1 Latency'] - df['L4 barrel 2 Latency']
df['Delta latency 2 (barrel 3 - barrel 2)'] = df['L4 barrel 3 Latency'] - df['L4 barrel 2 Latency']

# if either barrel 1 or barrel 2 has low SNR, don't compute the delta latency between these two
df.loc[(df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 1 (barrel 1 - barrel 2)'] = None

# same for barrel 2 or barrel 3
df.loc[(df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 2 (barrel 3 - barrel 2)'] = None

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                               (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 1 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 3 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[(df2['L4 barrel 1 SNR'] > snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_cross += len(df2[(df2['L4 barrel 2 SNR'] > snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] < snr_cutoff_signal)])
    n_barrel_none += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] < snr_cutoff_signal)])
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

In [ ]:
df['abs(delta 1)'] = np.abs(df['Delta latency 1 (barrel 1 - barrel 2)'])
df['abs(delta 2)'] = np.abs(df['Delta latency 2 (barrel 3 - barrel 2)'])



for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    dfd1 = df2['abs(delta 1)'].dropna()
    dfd2 = df2['abs(delta 2)'].dropna()

    print("Latency between barrel pairs in", stim_loc)
    dfds = np.concatenate([dfd1, dfd2])
    print("\t", 
          str(np.mean(dfds))[:5], 
          "+/-", 
          str(np.std(dfds))[:5], 
          "(n =", 
          str(len(dfds)) + ")")
    print("\t", dfds)

In [ ]:
# write the pandas dataframe back to csv
csv_filename = cell_type_prefix + '_all_slices-final.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [ ]:
df